<a href="https://colab.research.google.com/github/quiqueacla/IntrinsecValue_dcf/blob/devel/Intrinsec_Value_DCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Programa para facilitar la busqueda de datos para completar el excel de calculo de valor inrinseco de una compañía.
Seleccionar el TICKER de la compañia en la primera celda. Y ejecutar.
El programa busca en Yahoo Finance los valores para poder hacer copy Paste en el excel y facilitar el rellenado.

Luego en el EXCEL jugar con los parámetros restantes.

TODO: Hacer un jupiternotbook que automaticamente calcule el valor intrinseco.
Pero de todos modos finalmente hay que jugar con ciertos parámetros, puesto que a veces las compañias no se pueden valorar con los mismos parámetros.

In [1]:
!pip -q install yfinance pandas numpy
import locale
import yfinance as yf
import pandas as pd
import numpy as np



#pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
pd.set_option(
    "display.float_format",
    lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
)

TICKER = "IDR.MC"   # <-- cambia aquí si quieres otro
#TICKER = "VOW.DE"   # <-- cambia aquí si quieres otro
#TICKER = "MBG.DE"   # <-- cambia aquí si quieres otro
t = yf.Ticker(TICKER)


#Get General Data
info = t.info or {}
SharePrice  = info.get("currentPrice", np.nan)
Beta        = info.get("beta", np.nan)
NumShares   = info.get("sharesOutstanding", np.nan)
MarketCap = NumShares*SharePrice
TotalLiabilities = info.get("totalDebt", np.nan)
TotalAssets = info.get("totalAssets", np.nan)
DebtToEquity = info.get("debtToEquity", np.nan)

display(info)
#Datos para proyeccion FCFF

{'address1': 'Avenida de Bruselas, 35',
 'city': 'Alcobendas',
 'zip': '28108',
 'country': 'Spain',
 'phone': '34 91 480 50 00',
 'fax': '34 91 480 50 80',
 'website': 'https://www.indracompany.com',
 'industry': 'Information Technology Services',
 'industryKey': 'information-technology-services',
 'industryDisp': 'Information Technology Services',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Indra Sistemas, S.A. operates as a technology and consulting company for aerospace, defense, and mobility business worldwide. The company operates through the Defence, Air Traffic, Mobility, and Minsait segments. It offers transportation solutions and services under the Mova name, including Consulting for the planning and designing transport infrastructures; Collect for ticketing, tolling, and back office applications; Traffic for traffic management and control solutions; Protect for the protection of businesses and people; Experience,

In [2]:
##################################
# Data from Finantial statements
#################################

#Function to reorder columns
def tidy(df):
    if df is None or df.empty:
        return df
    out = df.copy()
    out.columns = pd.to_datetime(out.columns).date
    # ordenar columnas cronológicamente (ascendente)
    out = out.reindex(sorted(out.columns), axis=1)
    return out

#Funcion to pick a row from Table
def pick_row(df, names):
    if df is None or df.empty:
        return None
    idx = set(df.index.astype(str))
    for n in names:
        if n in idx:
            s = df.loc[n].astype(float)
            s.index = pd.to_datetime(s.index).date
            return s
    return None

#Get Finantial Statements
inc = t.financials
bal = t.balance_sheet
cf  = t.cashflow

#Order Tables
inc = tidy(inc)
bal = tidy(bal)
cf  = tidy(cf )

#Date form Incoming statement
Revenues = pick_row(inc, ["Total Revenue", "Revenue"])
NetIncome = pick_row(inc, ["Net Income", "Net Income Common Stockholders"])
NetIncomMargin = NetIncome / Revenues
InterestExpense = pick_row(inc, ["Interest Expense"])

EBIT = pick_row(inc, ["Ebit", "EBIT", "Operating Income", "OperatingIncome"])

IncomeTaxExpense = pick_row(inc, ["Tax Provision","Income Tax Expense"])
IncomeBeforeTax  = pick_row(inc, ["Income Before Tax", "Pretax Income"])
TaxRate          = IncomeTaxExpense / IncomeBeforeTax
TaxRate_mean     = TaxRate.mean()

Capex = pick_row(bal, ["Capital Expenditures"])

#Data from Balance
TotalDebt = pick_row(bal, ["Total Debt"])

TotalCurrentAssets  = pick_row(bal, ["Total Current Assets", "Current Assets"])
TotalCurrentLiabilities = pick_row(bal, ["Total Current Liabilities", "Current Liabilities"])
DeltaWorkingCapital = TotalCurrentAssets - TotalCurrentLiabilities

#Data from CashFlow
CFO = pick_row(cf, ["Operating Cash Flow"])
DA = pick_row(cf, ["Depreciation And Amortization"])
Capex = pick_row(cf, ["Capital Expenditures", "Capital Expenditure"])
Capex = -Capex;

###############################################
# BUILD Table to calculate TaxRate and CostLoan
###############################################
CostLoan = InterestExpense / TotalDebt
df_CostLoan_TaxRate = pd.DataFrame({
    "TotalDebt" : TotalDebt,
    "InterestExpense" : InterestExpense,
    "CostLoan" : CostLoan,
    "EBIT":EBIT,
    "IncomeBeforeTax":IncomeBeforeTax,
    "IncomeTaxExpense":IncomeTaxExpense,
    "TaxRate" : TaxRate,
})
display(df_CostLoan_TaxRate.T)

CostLoan_mean = CostLoan.mean()
print("--COST OF DEBT PARS--")
print(f"CostLoan  : {CostLoan_mean*100:.2f} %")
print(f"TaxRate   : {TaxRate_mean*100:.2f} %")
print("--COST OF EQUITY PARS--")
print("Beta      : ", Beta)
print("--MAIN INFORMATION--")
print("SharePrice: ", SharePrice)
print("NumShares : ", NumShares)



print("MarketCap : ", MarketCap)
print("--")
print("TotalLiabilities: ", TotalLiabilities)
print("TotalAssets: ", TotalAssets)
print("%Equity: ", MarketCap/(MarketCap+TotalLiabilities))
print("%Debt: ", TotalLiabilities/(MarketCap+TotalLiabilities))


###############################################
# BUILD Table to calculate Projections
###############################################
#Calculate FCFF
FCFF = CFO - Capex + InterestExpense*(1-TaxRate_mean)
#FCFF_1 = EBIT*(1-TaxRate_mean) + DA - Capex - DeltaWorkingCapital

df_Proyecciones_0 = pd.DataFrame({
    "Revenues" : Revenues,
    "NetIncome" : NetIncome,
    "NetIncomMargin" : NetIncomMargin,
    "CFO" : CFO,
    "Capex" : Capex,
    "DA" : DA,
    "InterestExpense":InterestExpense,
    "FCFF":FCFF,
})

print("--Tabla Proyecciones: Pasadas--")
display(df_Proyecciones_0.T)

# df1 = pd.DataFrame({
#     "EBIT":EBIT,
#     "IncomeBeforeTax":IncomeBeforeTax,
#     "IncomeTaxExpense":IncomeTaxExpense,
#     "TaxRate":TaxRate,
#     "CFO":CFO,
#     "Capex":Capex,
#     "DA":DA,
#     "TotalCurrentAssets":TotalCurrentAssets,
#     "TotalCurrentLiabilities":TotalCurrentLiabilities,
#     "DeltaWorkingCapital":DeltaWorkingCapital,
# })
# display(df1.T)






,2021-12-31,2022-12-31,2023-12-31,2024-12-31
TotalDebt,"1.723.047.000,00","1.239.291.000,00","1.036.543.000,00","850.874.000,00"
InterestExpense,"35.813.000,00","33.242.000,00","52.402.000,00","52.074.000,00"
CostLoan,"0,02","0,03","0,05","0,06"
EBIT,"247.254.000,00","289.003.000,00","355.994.000,00","438.335.000,00"
IncomeBeforeTax,"211.441.000,00","255.761.000,00","303.592.000,00","386.261.000,00"
IncomeTaxExpense,"65.408.000,00","80.172.000,00","94.896.000,00","105.981.000,00"
TaxRate,"0,31","0,31","0,31","0,27"


--COST OF DEBT PARS--
CostLoan  : 3.98 %
TaxRate   : 30.24 %
--COST OF EQUITY PARS--
Beta      :  0.663
--MAIN INFORMATION--
SharePrice:  54.6
NumShares :  175211845
MarketCap :  9566566737.0
--
TotalLiabilities:  717400000
TotalAssets:  nan
%Equity:  0.9302409256713254
%Debt:  0.06975907432867458
--Tabla Proyecciones: Pasadas--


,2021-12-31,2022-12-31,2023-12-31,2024-12-31
Revenues,"3.390.425.000,00","3.851.390.000,00","4.343.072.000,00","4.842.857.000,00"
NetIncome,"143.369.000,00","171.895.000,00","205.752.000,00","277.541.000,00"
NetIncomMargin,"0,04","0,04","0,05","0,06"
CFO,"344.636.000,00","350.261.000,00","379.521.000,00","450.895.000,00"
Capex,"49.156.000,00","54.119.000,00","62.707.000,00","111.000.000,00"
DA,"93.535.000,00","99.852.000,00","99.070.000,00","106.950.000,00"
InterestExpense,"35.813.000,00","33.242.000,00","52.402.000,00","52.074.000,00"
FCFF,"320.461.693,36","319.330.268,25","353.367.505,58","376.219.706,11"


A partir de aquí vamos a sacar el Valor intrinseco

In [18]:
#############################################################
# Calculo de Proyecciones
# En base a los 4 años pasados, y las estimaciones de analistas
# para y0 y y1, proyectamos FFCF para y0,y1,y2,y3,y4
#############################################################
#PARS
TerminalGrowth_neutro = 0.02
WAC_neutro_typ = 0.09#WACC

RiskFreeRate = 0.029    #Bono soberano 10y
EquityRiskPrime = 0.07  # mercados EUR
CostEquity = RiskFreeRate + EquityRiskPrime * Beta

CostLoan = CostLoan_mean
CostDebt = CostLoan *(1-TaxRate_mean)

E_p = 1 / (DebtToEquity/100 + 1)
D_p = 1-E_p

#Seleccionar el WACC calculado de balance o el typico de parametro
#WAC_neutro = E_p * CostEquity + D_p * CostDebt
WAC_neutro = WAC_neutro_typ

#Revenues Estimadas de los analista para y0 y1
rev_est = t.revenue_estimate
re_y0 = rev_est.loc["0y","avg"]
re_y1 = rev_est.loc["+1y","avg"]

#Generamos el array de ventas pasadas y estimadas y sacamos el Growth
Revenues = pick_row(inc, ["Total Revenue", "Revenue"])
Revenues = np.array(Revenues)
Revenues_next = np.array([re_y0, re_y1])
Revenues = np.concatenate([Revenues, Revenues_next])

Growth = (Revenues[1:] - Revenues[:-1]) / Revenues[:-1]
Growth_mean = Growth.mean()
Growth_std = Growth.std()
#Con el crecimiento medio estimamos los años siguientes y2 y3 y4
tmp = []
tmp.append(Revenues[-1]*(1+Growth_mean))
tmp.append(Revenues[-1]*(1+Growth_mean)**2)
tmp.append(Revenues[-1]*(1+Growth_mean)**3)
Revenues = np.concatenate([Revenues, tmp])
Growth = (Revenues[1:] - Revenues[:-1]) / Revenues[:-1]

#Calculamos el NetIncome y NetIncomMargin
#NetIncome = Revenues*NetIncomMargin
NetIncome = pick_row(inc, ["Net Income", "Net Income Common Stockholders"])

NetIncomeMargin = np.array(NetIncome[0:4])/np.array(Revenues[0:4])
NetIncomeMargin_mean = NetIncomeMargin.mean()
NetIncomeMargin = np.append(NetIncomeMargin, [NetIncomeMargin_mean]*5)

NetIncome = Revenues * NetIncomeMargin



#CashFlow, Capex e Intereses para sacat FFCF
CFO = pick_row(cf, ["Operating Cash Flow"])
Capex = pick_row(cf, ["Capital Expenditures", "Capital Expenditure"])
InterestExpense = pick_row(inc, ["Interest Expense"])

CFO=np.array(CFO)
Capex=np.array(Capex)
Capex=-Capex
InterestExpense=np.array(InterestExpense)

FCF_gen = CFO - Capex
FCFF    = CFO - Capex + InterestExpense*(1-TaxRate_mean)

FCFF_NI_ratio = FCFF/NetIncome[:4]
FCFF_NI_ratio_mean = FCFF_NI_ratio.mean()
FCFF_NI_ratio = np.append(FCFF_NI_ratio, [FCFF_NI_ratio_mean]*5)
FCFF = NetIncome * FCFF_NI_ratio #Machacamos los 4 primeros pero no pasa nada porque da lo mismo

################################################################
# Presentación Parámetros y protecciones
################################################################
BOLD = "\033[1m"
RESET = "\033[0m"
print(f"\n{BOLD} PARÁMETROS COMUNES y PROYECCIONES     {RESET}")
print(f"{BOLD}-----------------------------------------{RESET}")
#CostEquity = RiskFreeRate + EquityRiskPrime * Beta
print(f"RiskFreeRate (Rf):\t{RiskFreeRate*100:.2f}% (from Program Constants)")
print(f"EquityPrime (EPR):\t{EquityRiskPrime*100:.2f}% (from Program Constants)")
print(f"Beta             :\t{Beta:.2f}")
print(f"CostEquity   (Ke):\t{CostEquity*100:.2f}%")

#CostDebt = CostLoan *(1-TaxRate_mean)
print(f"CostLoan     (Cl):\t{CostLoan*100:.2f}% (from IncomeStatement)")
print(f"TaxRate_mean  (T):\t{TaxRate_mean*100:.2f}% (from IncomeStatement)")
print(f"CostDebt     (Kd):\t{CostDebt*100:.2f}%")

print(f"Debt/Equity (D/E):\t{DebtToEquity:.2f}% (from yfinance variable)")
print(f"Equity/Assets (E):\t{E_p*100:.2f}%")
print(f"Debt/Assets   (D):\t{D_p*100:.2f}%")
print(f"{BOLD}WACC_neutro:\t{WAC_neutro*100:.2f}%{RESET} (Typ {WAC_neutro_typ*100:.2f}%)= (E/(E+D) x Ke + (D/(E+D) x Kd))")
print(f"{BOLD}Tg_neutro  :\t{TerminalGrowth_neutro*100:.2f}%{RESET}")

#Fill columns not used to print table with panda
CFO = np.append(CFO, [-1]*5)
Capex = np.append(Capex, [-1]*5)
InterestExpense = np.append(InterestExpense, [-1]*5)
FCF_gen = np.append(FCF_gen, [-1]*5)
Growth =np.append(Growth, [-1])

df_Proyecciones_1 = pd.DataFrame({
    "Revenues" : Revenues,
    "Growth" : Growth,
    "NetIncome" : NetIncome,
    "NetIncomMargin" : NetIncomeMargin,
    "CFO" : CFO,
    "Capex" : Capex,
    "FCFF_gen":FCF_gen,
    "InterestExpense":InterestExpense,
    "FCFF":FCFF,
})
df_Proyecciones_1_Transpuesta = df_Proyecciones_1.T
df_Proyecciones_1_Transpuesta.columns = ["-4y", "-3y", "-2y", "-1y", "y0", "+1y", "+2y", "+3y", "+4y"]
display(df_Proyecciones_1_Transpuesta)


#########################################################
# Calculo de Escenarios
#########################################################
scenarios = [
    {"name": "Bajista",  "delta_wacc": +0.02, "delta_tg": -0.01},
    {"name": "Neutro",   "delta_wacc":  0.00, "delta_tg":  0.000},
    {"name": "Alcista",  "delta_wacc": -0.02, "delta_tg": +0.01},
]

Summary={}

for s in scenarios:

    Tg   = TerminalGrowth_neutro + s["delta_tg"]
    WACC = WAC_neutro + s["delta_wacc"]

    # print(
    #     f"{s['name']:8s} | "
    #     f"WACC={WACC*100:.2f}% | "
    #     f"g={Tg*100:.2f}%"
    # )


    FCFF_plus_TerminalValue = FCFF[-5:-1] #Cogemos y0,y1,y2,y3 (y4 lo dejamos)
    TerminalValue = FCFF_plus_TerminalValue[-1]*(1+Tg)/(WACC-Tg)

    FCFF_plus_TerminalValue = np.append(FCFF_plus_TerminalValue, TerminalValue)
    DiscountFactor = 1/(1+WACC)**np.arange(1,6)
    DiscountFactor[-1] = DiscountFactor[-2]
    PresentValue = FCFF_plus_TerminalValue * DiscountFactor

    ######################################################
    # Presentacion
    ######################################################
    print(f"\n{BOLD} Análisis ESCENARIO {s['name'].upper()}{RESET}")
    print(f"{BOLD}-----------------------------------------{RESET}")
    print(f"{BOLD}WACC             :\t{WACC*100:.2f}%{RESET}= (E/(E+D) x Ke + (D/(E+D) x Kd))")
    print(f"{BOLD}Terminal Growth  :\t{Tg*100:.2f}%{RESET}")
    print()

    f"Scenario: {s['name']}"
    title = f"<b>FCFF para {TICKER}. Escenario Scenario: {s['name']}:\nWACC {WACC*100:.2f}%  Growth {Tg*100:.2f}</b>"

    df_FCFF_Neutro = pd.DataFrame({
        "FCFF_plus_TerminalValue":FCFF_plus_TerminalValue,
        "DiscountFactor":DiscountFactor,
        "PresentValue":PresentValue,
    })
    df_FCFF_Neutro_Transpuesta = df_FCFF_Neutro.T
    df_FCFF_Neutro_Transpuesta.columns = ["y0", "y1", "y2", "y3", "TerminalValue"]
    df_FCFF_Neutro_Transpuesta.style.set_caption(title)
    display(df_FCFF_Neutro_Transpuesta.style.set_caption(title))


    ValorIntrinsec_per_share = PresentValue.sum()/NumShares
    print("")
    print("")
    title = f"<b>Valor Intrinseco {TICKER}. Valor Intrinseco</b>"
    df_VI_Neutro = pd.DataFrame({
        "Valor intrinseco": [PresentValue.sum()],
        "SharesOut": [NumShares],
        "Valor intrinseco por accion": [ValorIntrinsec_per_share],
    })
    display(df_VI_Neutro.T.style.set_caption(title))


    Summary[s["name"]] = {
        "WACC":WACC,
        "Tg":Tg,
        "CurrentPrice":SharePrice,
        "IntrinsecValue":ValorIntrinsec_per_share,
        "Margin":(ValorIntrinsec_per_share/SharePrice - 1)
    }



df = pd.DataFrame(Summary)

df["Mean"] = df.mean(axis=1)

rows_percent = ["Tg", "WACC", "Margin"]
rows_perun   = ["CurrentPrice", "IntrinsecValue"]

idx = pd.IndexSlice

display(
    df.style
    .format("{:.2%}", subset=idx[rows_percent, :])
    .format("{:,.2f} €", subset=idx[rows_perun, :])
    .set_caption(f"Escenarios DCF – {TICKER}")
)








 PARÁMETROS COMUNES y PROYECCIONES     
-----------------------------------------
RiskFreeRate (Rf):	2.90% (from Program Constants)
EquityPrime (EPR):	7.00% (from Program Constants)
Beta             :	0.66
CostEquity   (Ke):	7.54%
CostLoan     (Cl):	3.98% (from IncomeStatement)
TaxRate_mean  (T):	30.24% (from IncomeStatement)
CostDebt     (Kd):	2.78%
Debt/Equity (D/E):	47.21% (from yfinance variable)
Equity/Assets (E):	67.93%
Debt/Assets   (D):	32.07%
WACC_neutro:	9.00% (Typ 9.00%)= (E/(E+D) x Ke + (D/(E+D) x Kd))
Tg_neutro  :	2.00%


,-4y,-3y,-2y,-1y,y0,+1y,+2y,+3y,+4y
Revenues,"3.390.425.000,00","3.851.390.000,00","4.343.072.000,00","4.842.857.000,00","5.327.036.810,00","6.265.482.710,00","7.086.067.281,29","8.014.123.067,45","9.063.725.475,74"
Growth,"0,14","0,13","0,12","0,10","0,18","0,13","0,13","0,13","-1,00"
NetIncome,"143.369.000,00","171.895.000,00","205.752.000,00","277.541.000,00","255.168.380,98","300.120.524,07","339.427.036,11","383.881.486,28","434.158.095,35"
NetIncomMargin,"0,04","0,04","0,05","0,06","0,05","0,05","0,05","0,05","0,05"
CFO,"344.636.000,00","350.261.000,00","379.521.000,00","450.895.000,00","-1,00","-1,00","-1,00","-1,00","-1,00"
Capex,"49.156.000,00","54.119.000,00","62.707.000,00","111.000.000,00","-1,00","-1,00","-1,00","-1,00","-1,00"
FCFF_gen,"295.480.000,00","296.142.000,00","316.814.000,00","339.895.000,00","-1,00","-1,00","-1,00","-1,00","-1,00"
InterestExpense,"35.813.000,00","33.242.000,00","52.402.000,00","52.074.000,00","-1,00","-1,00","-1,00","-1,00","-1,00"
FCFF,"320.461.693,36","319.330.268,25","353.367.505,58","376.219.706,11","457.128.967,81","537.659.818,44","608.076.635,79","687.715.879,66","777.785.402,85"



 Análisis ESCENARIO BAJISTA
-----------------------------------------
WACC             :	11.00%= (E/(E+D) x Ke + (D/(E+D) x Kd))
Terminal Growth  :	1.00%



,y0,y1,y2,y3,TerminalValue
FCFF_plus_TerminalValue,457128967.812514,537659818.436557,608076635.788056,687715879.655140,6945930384.516918
DiscountFactor,0.900901,0.811622,0.731191,0.658731,0.658731
PresentValue,411827898.930193,436376770.097035,444620395.258703,453019751.340216,4575499488.536185


,0
Valor intrinseco,6321344304.162333
SharesOut,175211845.000000
Valor intrinseco por accion,36.078293


...

,0
Valor intrinseco,8927794922.773121
SharesOut,175211845.000000
Valor intrinseco por accion,50.954289



 Análisis ESCENARIO ALCISTA
-----------------------------------------
WACC             :	7.00%= (E/(E+D) x Ke + (D/(E+D) x Kd))
Terminal Growth  :	3.00%



,y0,y1,y2,y3,TerminalValue
FCFF_plus_TerminalValue,457128967.812514,537659818.436557,608076635.788056,687715879.655140,17708683901.119869
DiscountFactor,0.934579,0.873439,0.816298,0.762895,0.762895
PresentValue,427223334.404219,469612908.058832,496371666.780722,524655151.837959,13509870159.827436


,0
Valor intrinseco,15427733220.909168
SharesOut,175211845.000000
Valor intrinseco por accion,88.051885


,Bajista,Neutro,Alcista,Mean
WACC,11.00%,9.00%,7.00%,9.00%
Tg,1.00%,2.00%,3.00%,2.00%
CurrentPrice,54.60 €,54.60 €,54.60 €,54.60 €
IntrinsecValue,36.08 €,50.95 €,88.05 €,58.36 €
Margin,-33.92%,-6.68%,61.27%,6.89%
